**Корректность проверена на Python 3.6:**
+ pandas 0.23.4
+ numpy 1.15.4
+ sklearn 0.20.2

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Sklearn

## sklearn.grid_search

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [2]:
from sklearn import model_selection, datasets, linear_model, metrics

import numpy as np
import pandas as pd

### Генерация датасета

In [3]:
iris = datasets.load_iris()

In [4]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                     test_size = 0.3,random_state = 0)

### Задание модели

In [5]:
classifier = linear_model.SGDClassifier(random_state = 0, tol=1e-3)

### Генерация сетки

In [6]:
classifier.get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [7]:
parameters_grid = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    'max_iter' : np.arange(5,10),
    'alpha' : np.linspace(0.0001, 0.001, num = 5),
}

In [8]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 0)

### Подбор параметров и оценка качества

#### Grid search

In [9]:
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

In [10]:
%%time
grid_cv.fit(train_data, train_labels)

CPU times: user 5.91 s, sys: 44 ms, total: 5.95 s
Wall time: 5.96 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             estimator=SGDClassifier(random_state=0),
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'loss': ['hinge', 'log', 'squared_hinge',
                                  'squared_loss'],
                         'max_iter': array([5, 6, 7, 8, 9]),
                         'penalty': ['l1', 'l2']},
             scoring='accuracy')

In [11]:
grid_cv.best_estimator_

SGDClassifier(alpha=0.0007750000000000001, max_iter=9, penalty='l1',
              random_state=0)

In [12]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

0.9047619047619048
{'alpha': 0.0007750000000000001, 'loss': 'hinge', 'max_iter': 9, 'penalty': 'l1'}


In [13]:
grid_cv.cv_results_

{'mean_fit_time': array([0.00295441, 0.00212858, 0.00215442, 0.00223856, 0.0022512 ,
        0.00224729, 0.00231471, 0.00225985, 0.00233169, 0.00226524,
        0.00228271, 0.00219064, 0.00219486, 0.00218523, 0.00224292,
        0.00221684, 0.00229087, 0.00223296, 0.00232501, 0.00234377,
        0.00226884, 0.00218065, 0.00217636, 0.00215006, 0.00218756,
        0.00229211, 0.00224338, 0.00219471, 0.00226216, 0.00231435,
        0.00228271, 0.00232902, 0.00243192, 0.00224788, 0.002263  ,
        0.00225813, 0.0025162 , 0.00229845, 0.00232995, 0.00218065,
        0.00215688, 0.00211923, 0.00216289, 0.00212798, 0.00215006,
        0.00212028, 0.00220249, 0.0021663 , 0.00217206, 0.00214245,
        0.00220082, 0.00217981, 0.00225365, 0.00233381, 0.00235667,
        0.0023241 , 0.00236061, 0.0023227 , 0.00232294, 0.00223   ,
        0.00215364, 0.00211906, 0.00209322, 0.00204153, 0.00208063,
        0.00204585, 0.00209961, 0.00206532, 0.00215199, 0.00213041,
        0.00206842, 0.00206935,

#### Randomized grid search

In [14]:
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv, n_iter = 20, 
                                                   random_state = 0)

In [15]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

CPU times: user 624 ms, sys: 4.2 ms, total: 628 ms
Wall time: 626 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
                   estimator=SGDClassifier(random_state=0), n_iter=20,
                   param_distributions={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                                        'loss': ['hinge', 'log',
                                                 'squared_hinge',
                                                 'squared_loss'],
                                        'max_iter': array([5, 6, 7, 8, 9]),
                                        'penalty': ['l1', 'l2']},
                   random_state=0, scoring='accuracy')

In [17]:
print(randomized_grid_cv.best_score_)
print(randomized_grid_cv.best_params_)

0.8666666666666666
{'penalty': 'l1', 'max_iter': 8, 'loss': 'squared_hinge', 'alpha': 0.0007750000000000001}


In [18]:
#!/usr/bin/python

from datetime import datetime


def control_sum(str):
    return sum(map(ord, list(str)))


def check(reply):
    def _is_number(str):
        try:
            int(str)
            return True
        except ValueError:
            return False

    if "Control sum:" not in reply:
        return False
    parts = reply.split("Control sum:")
    received_current_time = parts[0].strip()
    received_control_sum = parts[1].strip()
    if not _is_number(received_control_sum):
        return False
    else:
        received_control_sum = int(received_control_sum)
    expected_control_sum = control_sum(received_current_time)
    return expected_control_sum == received_control_sum


current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(current_time)
print("Control sum: " + str(control_sum(current_time)))

#print check(current_time + '\n' + "Control sum: " + str(control_sum(current_time)))

2021-03-16 12:03:36
Control sum: 940
